In [1]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [2]:
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


/u/nneoma/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
dropped columns


/tmp/ipykernel_851781/1199118304.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned color
cleaned breed
Done running ml_project.ipynb.


In [3]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
#from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

In [4]:
df_train = time_as_cyclical(df_train)
#df_train = bucket_days(df_train)

df_test = time_as_cyclical(df_test)
#df_test = bucket_days(df_test)

In [ ]:
df_train = df_train.drop(columns=['size'])
df_test = df_test.drop(columns=['size'])
df_train.head()

,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,intake_year,primary_color,is_mix,size,outcome_type,hour_sin,hour_cos,month_sin,month_cos
0,Stray,Normal / Behavior,Dog,Spayed Female,96.0,2,2015,white,0,3,Return to Owner,1.224647e-16,-1.000000e+00,-0.500000,-0.866025
1,Stray,Normal / Behavior,Dog,Intact Male,11.0,7,2016,sable,1,2,Return to Owner,-1.000000e+00,-1.836970e-16,0.866025,-0.500000
2,Public Assist,Normal / Behavior,Cat,Neutered Male,24.0,Common,2022,orange,0,2,Transfer,0.000000e+00,1.000000e+00,0.500000,-0.866025
3,Owner Surrender,Normal / Behavior,Dog,Neutered Male,24.0,2,2017,chocolate,1,4,Return to Owner,1.224647e-16,-1.000000e+00,0.866025,0.500000
4,Public Assist,Normal / Behavior,Dog,Neutered Male,72.0,3,2019,black,1,5,Return to Owner,7.071068e-01,-7.071068e-01,0.866025,-0.500000


In [ ]:
# {'logreg__penalty': 'l2', 'logreg__fit_intercept': False, 'logreg__class_weight': 'balanced', 'logreg__C': 0.01}

def train_logreg_classifier(X_train, y_train, X_test, rare_classes):
    cat_cols = ['intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed','primary_color','is_mix']
    num_cols = ['age_upon_intake', 'intake_year']    # Replace with your actual numerical columns


    class_labels = np.unique(y_train)  # Get unique class labels in your target variable
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=class_labels,
        y=y_train
    )
    class_weight_dict = dict(zip(class_labels, class_weights))  # Map class labels to weights

    # Handle rare categories
    rare_threshold = 0.01
    for col in cat_cols:
        freq = X_train[col].value_counts(normalize=True)
        rare_categories = freq[freq < rare_threshold].index
        X_train[col] = X_train[col].apply(lambda x: 'Other' if x in rare_categories else x)
        X_test[col] = X_test[col].apply(lambda x: 'Other' if x in rare_categories else x)

    for col in cat_cols:
        X_train[col] = X_train[col].astype(str)
        X_test[col] = X_test[col].astype(str)


    # Preprocessing for numerical features: Imputation (if needed) + Standardization
    numerical_transformer = Pipeline(steps=[
    #    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values (if any)
        ('scaler', StandardScaler())  # Standardize numerical features
    ])

    # Preprocessing for categorical features: Imputation (if needed) + One-Hot Encoding
    categorical_transformer = Pipeline(steps=[
     #   ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values (if any)
        ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-Hot Encode categorical features
    ])

    # Combine both transformations into a single ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ])

    # Create a pipeline that first transforms the data and then applies Logistic Regression
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('logreg', LogisticRegression(max_iter=1000))  # You can adjust max_iter as needed
    ])

    param_distributions = {
        "logreg__C": [0.001, 0.01, 0.1, 1, 10, 100], #uniform(loc=0.01, scale=10),
        "logreg__penalty": ['l2', 'l1'],
        'logreg__solver' : ['lbfgs','liblinear'],
        "logreg__fit_intercept": [True, False],
        'logreg__class_weight': [class_weight_dict, 'balanced'],
        'logreg__max_iter' : [1000, 3000, 5000]
    }

    randomized_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_distributions,
        n_iter=50,
        cv=skf,
        scoring=balanced_acc_scorer,
        verbose=3,
        random_state=42
    )

    randomized_search.fit(X_train, y_train)

    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)

    cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=skf, verbose=3, scoring=balanced_acc_scorer)
    print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

    test_predictions = randomized_search.predict(X_test)

    return randomized_search.best_estimator_, test_predictions

In [7]:
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']

X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20, random_state=42)

In [ ]:
# Encode the target variable.
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
print('Encoding mapping:', le.classes_)

# Identify rare classes that need oversampling.
rare_classes = [
    label for label, count in pd.Series(y_train).value_counts().items()
    if count < 0.05 * len(y_train)
]
print("Rare classes:")
for cls in rare_classes:
    print(f"  {cls}: {le.classes_[cls]}")

# Define column groups (this example will one-hot encode all columns, so cat_cols and num_cols are not used in the transformer).
cat_cols = {'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'intake_hour', 'intake_year', 'primary_color', 'season', 'is_mix', 'age_group'}
categorical_features = [col for col in X_train.columns if col in cat_cols]


'''
# cat_cols_freq = ['primary_color']

# Frequency encode selected high-cardinality features
for col in cat_cols_freq:
  freq_map = X_train[col].value_counts()
  X_train[col] = X_train[col].map(freq_map)
  X_test[col]  = X_test[col].map(freq_map).fillna(0)
  '''

# Train the classifier with the refactored pipeline.


best_model, test_predictions = train_logreg_classifier(
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    rare_classes=rare_classes
)

# Convert predictions back to original labels.
classification_report_with_accuracy_score(y_test, test_predictions)

Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Rare classes:
  2: Euthanasia
  1: Died
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END logreg__C=0.001, logreg__class_weight=balanced, logreg__fit_intercept=False, logreg__max_iter=5000, logreg__penalty=l2, logreg__solver=liblinear;, score=0.437 total time=   0.2s
[CV 2/5] END logreg__C=0.001, logreg__class_weight=balanced, logreg__fit_intercept=False, logreg__max_iter=5000, logreg__penalty=l2, logreg__solver=liblinear;, score=0.432 total time=   0.2s
[CV 3/5] END logreg__C=0.001, logreg__class_weight=balanced, logreg__fit_intercept=False, logreg__max_iter=5000, logreg__penalty=l2, logreg__solver=liblinear;, score=0.437 total time=   0.2s
[CV 4/5] END logreg__C=0.001, logreg__class_weight=balanced, logreg__fit_intercept=False, logreg__max_iter=5000, logreg__penalty=l2, logreg__solver=liblinear;, score=0.447 total time=   0.2s
[CV 5/5] END logreg__C=0.001, logreg__class_weight=bal